In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

def get_foreign_trading_data(start_date, end_date, exchange, time_frame):
    """
    Lấy dữ liệu giao dịch của nước ngoài
    
    Args:
        start_date (str): Ngày bắt đầu (format: 'YYYY-MM-DD')
        end_date (str): Ngày kết thúc (format: 'YYYY-MM-DD')
        exchange (str): Sàn giao dịch ('HOSE', 'HNX', 'UPCOM')
        time_frame (str): Khung thời gian ('ONE_DAY', 'ONE_WEEK', 'ONE_MONTH')
    
    Returns:
        pd.DataFrame: DataFrame chứa dữ liệu giao dịch nước ngoài
    """
    
    # Chuyển đổi ngày sang epoch timestamp
    start_date_epoch = int(pd.to_datetime(start_date).timestamp())
    end_date_epoch = int(pd.to_datetime(end_date).timestamp())
    

    url = "https://trading.vietcap.com.vn/api/market-watch/v3/ForeignVolumeChart/getAll"
    
    payload = json.dumps({
        "from": start_date_epoch,
        "to": end_date_epoch,
        "group": exchange,
        "timeFrame": f"ONE_{time_frame.upper()}"
    })
    
    headers = {
        'Accept': 'application/json',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,ko;q=0.7,fr;q=0.6,zh-TW;q=0.5,zh;q=0.4',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        'Origin': 'https://trading.vietcap.com.vn',
        'Referer': 'https://trading.vietcap.com.vn/iq/market?type=stock',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        
        data = pd.DataFrame(response.json())
        
        if data.empty:
            return data
        
        # Xử lý dữ liệu
        data = data.rename(columns={"truncTime": "time"})
        data['time'] = pd.to_datetime(data['time'], unit='s').dt.date
        
        # Chuyển đổi các cột thành số
        numeric_columns = ['foreignBuyVolume', 'foreignSellVolume', 'foreignBuyValue', 'foreignSellValue']
        for col in numeric_columns:
            data[col] = pd.to_numeric(data[col])
        
        # Tính toán các chỉ số bổ sung
        data['net_value'] = data['foreignBuyValue'] - data['foreignSellValue']
        data['accumulated_net_value'] = data['net_value'].cumsum()
        
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gọi API: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Lỗi xử lý dữ liệu: {e}")
        return pd.DataFrame()

# # Ví dụ sử dụng
# if __name__ == "__main__":


In [ ]:
# Lấy dữ liệu từ đầu năm đến hiện tại
start_date = '2025-01-02'
end_date = datetime.now().strftime('%Y-%m-%d')
    
data = get_foreign_trading_data(start_date, end_date, "HOSE", "day")

In [3]:
# Cài đặt pyecharts nếu chưa có
# !pip install pyecharts

from pyecharts import options as opts
from pyecharts.charts import Bar, Line
from pyecharts.commons.utils import JsCode

# Chuẩn bị dữ liệu
dates = [str(date) for date in data['time']]
net_values = (data['net_value'] / 1e9).tolist()
accumulated_values = (data['accumulated_net_value'] / 1e9).tolist()

# Tạo biểu đồ cột
bar = (
    Bar(init_opts=opts.InitOpts(bg_color="#1e1e1e", width="900px", height="450px"))
    .add_xaxis(dates)
    .add_yaxis(
        "Giá trị ròng",
        net_values,
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(
            color=JsCode("""
                function(params) {
                    return params.value >= 0 ? '#00da3c' : '#ec0000';
                }
            """)
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="Giá trị ròng (tỷ VND)",
            position="right",
            axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color="#666")),
            axislabel_opts=opts.LabelOpts(color="#999"),
            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(color="#333", opacity=0.3))
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Giao dịch khối ngoại HOSE", title_textstyle_opts=opts.TextStyleOpts(color="#fff")),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=-45, color="#999"),
            axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color="#666")),
            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(color="#333", opacity=0.3))
        ),
        yaxis_opts=opts.AxisOpts(
            name="Giá trị tích lũy (tỷ VND)",
            position="left",
            axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color="#666")),
            axislabel_opts=opts.LabelOpts(color="#999"),
            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(color="#333", opacity=0.3))
        ),
        datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=100)],
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color="#fff"))
    )
)

# Tạo biểu đồ đường
line = (
    Line()
    .add_xaxis(dates)
    .add_yaxis(
        "Giá trị tích lũy",
        accumulated_values,
        yaxis_index=0,
        label_opts=opts.LabelOpts(is_show=False),
        linestyle_opts=opts.LineStyleOpts(color="#1f77b4", width=2),
        itemstyle_opts=opts.ItemStyleOpts(color="#1f77b4")
    )
)

# Kết hợp biểu đồ
chart = bar.overlap(line)
chart.render_notebook()